In [1]:
import numpy as np
import torch
import time

In [2]:
x = torch.randn(20000,300)
y = torch.randn(300,20000)
start = time.time()
torch.mm(x,y)
end = time.time()
print(f'CPU: {end-start}')

device = torch.device('cuda')
torch.cuda.get_device_name(device)
x.to(device)
y.to(device)
start = time.time()
torch.mm(x,y)
end = time.time()

CPU: 0.9536142349243164


In [3]:
print(f'GPU: {end-start}')
print(y.t())
print(y)

GPU: 0.7574074268341064
tensor([[ 0.4076, -1.0656,  0.6823,  ..., -0.4193, -1.0232,  1.2137],
        [ 1.4242,  1.2847,  1.2126,  ...,  0.1277, -0.5334,  0.6845],
        [ 1.9417, -1.2490, -0.3607,  ..., -0.5175, -0.3752, -0.7473],
        ...,
        [ 1.4075, -0.7679, -0.8760,  ...,  0.1625,  0.3926,  0.0974],
        [ 0.0466, -0.6632, -0.4482,  ..., -0.1491,  1.3546,  0.0722],
        [-0.0453, -1.0134,  0.2988,  ..., -1.3858, -1.3069,  0.8617]])
tensor([[ 0.4076,  1.4242,  1.9417,  ...,  1.4075,  0.0466, -0.0453],
        [-1.0656,  1.2847, -1.2490,  ..., -0.7679, -0.6632, -1.0134],
        [ 0.6823,  1.2126, -0.3607,  ..., -0.8760, -0.4482,  0.2988],
        ...,
        [-0.4193,  0.1277, -0.5175,  ...,  0.1625, -0.1491, -1.3858],
        [-1.0232, -0.5334, -0.3752,  ...,  0.3926,  1.3546, -1.3069],
        [ 1.2137,  0.6845, -0.7473,  ...,  0.0974,  0.0722,  0.8617]])


In [4]:
dtype = torch.float

batch, input, hidden, out = 64, 100, 250, 10
lr, epochs = 1e-6, 500

x = torch.randn(batch, input, device=device, dtype=dtype)
y = torch.randn(batch, out, device=device, dtype=dtype)

w1 = torch.randn(input, hidden, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(hidden, out, device=device, dtype=dtype, requires_grad=True)

In [5]:
for t in range(epochs):
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    e = (y_pred - y)
    loss = (y_pred - y).pow(2).sum()
    if not t%100:
        print(f'Epoch: {t} mse: {loss.item()} mae: {loss.item()/batch}')
    
    loss.backward()
    
    with torch.no_grad():
        w1 -= lr * w1.grad
        w2 -= lr * w2.grad
        
        w1.grad.zero_()
        w2.grad.zero_()

Epoch: 0 mse: 9405384.0 mae: 146959.125
Epoch: 100 mse: 24917.833984375 mae: 389.3411560058594
Epoch: 200 mse: 1230.0728759765625 mae: 19.21988868713379
Epoch: 300 mse: 91.27224731445312 mae: 1.42612886428833
Epoch: 400 mse: 8.209875106811523 mae: 0.12827929854393005
